In [1]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn
%matplotlib inline

In [2]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [3]:
batch_size = 512
pred_len = 60

In [4]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_1.csv').fillna(0)

In [5]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [6]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [7]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

Its crucial that the `testloader` is not shuffled, as we'll use the page names directly from the train csv to index them later.

In [8]:
testloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:,:]).float(),
        torch.zeros(X.shape[0])
    ),
    batch_size=batch_size, shuffle=False
)

In [9]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_v2.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=20, save_best_path=save_best_path)


EPOCH 1
Running average loss: 0.380938
VALIDATION LOSS: 0.456770
Elapsed time 161.36574244499207 seconds

EPOCH 2
Running average loss: 0.387845
VALIDATION LOSS: 0.499010
Elapsed time 161.3883695602417 seconds

EPOCH 3


In [ ]:
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=5)


EPOCH 1
Running average loss: 0.323598
VALIDATION LOSS: 0.419218
Elapsed time 161.98347640037537 seconds

EPOCH 2
Running average loss: 0.322527
VALIDATION LOSS: 0.419451
Elapsed time 160.9306697845459 seconds

EPOCH 3
Running average loss: 0.321289
VALIDATION LOSS: 0.412642
Elapsed time 160.79859447479248 seconds

EPOCH 4
Running average loss: 0.319998
VALIDATION LOSS: 0.418535
Elapsed time 160.88264060020447 seconds

EPOCH 5
Running average loss: 0.320067
VALIDATION LOSS: 0.413388
Elapsed time 161.06813526153564 seconds
Elapsed time 805.6670889854431 seconds
